<a href="https://colab.research.google.com/github/ParalelaUnsaac/G4-2020-1/blob/main/Copy_of_roxana_mpi_para_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Distributed Parallel Programming Patterns using mpi4py
Libby Shoop, Macalester College

¡Bienvenido!

Este libro contiene algunos ejemplos que ilustran los conceptos fundamentales básicos de la computación distribuida utilizando código Python. El tipo de computación que ilustran estos ejemplos se llama * paso de mensajes *. El paso de mensajes es una forma de programación que se basa en procesos que se comunican entre sí para coordinar su trabajo. El paso de mensajes se puede utilizar en una sola computadora multinúcleo o con un grupo de computadoras.


### Patrones de software

Los patrones en el software son implementaciones comunes que los profesionales han utilizado una y otra vez para realizar tareas. A medida que los profesionales los usan repetidamente, la comunidad comienza a darles nombres y catalogarlos, convirtiéndolos a menudo en funciones de biblioteca reutilizables. Los ejemplos que verá en este libro se basan en patrones documentados que se han utilizado para resolver diferentes problemas mediante el paso de mensajes entre procesos. El paso de mensajes es una forma de computación distribuida que usa procesos, que se pueden usar en grupos de computadoras o máquinas multinúcleo.

En muchos de estos ejemplos, el nombre del patrón es parte del nombre del archivo de código Python. También verá que, a menudo, las funciones de la biblioteca MPI también toman el nombre del patrón, y la implementación de esas funciones contiene el patrón que los profesionales utilizan con frecuencia. Estos ejemplos de código de patrón que le mostramos aquí, denominados patternlets, se basan en el trabajo original de Joel Adams:

Adams, Joel C. "Patternlets: una herramienta de enseñanza para presentar a los estudiantes los patrones de diseño paralelos". Taller del Simposio Internacional de Procesamiento Distribuido y Paralelo del IEEE 2015. IEEE, 2015.

Para ejecutar estos ejemplos, primero necesitará instalar la biblioteca mpi4py ejecutando este código (esto generalmente llevará un tiempo instalarlo la primera vez):

In [1]:
! pip install mpi4py

     |████████████████████████████████| 1.4MB 3.2MB/s 
  Created wheel for mpi4py: filename=mpi4py-3.0.3-cp36-cp36m-linux_x86_64.whl size=2074496 sha256=45dcf1c92f032d7cccbaaae87b0ccc6bd6b08aba1a5577154fcf31abb79f52ea
  Stored in directory: /root/.cache/pip/wheels/18/e0/86/2b713dd512199096012ceca61429e12b960888de59818871d6
Successfully built mpi4py


![Important symbol](https://drive.google.com/uc?id=1AWRLAqeaqi7SG7PHyOVywZRuMDK9Z2_s)**Important:** 
Importante: tendrá que volver a ejecutar esta celda después de desconectarse durante bastante tiempo.

# Patrones de estructura del programa

> Bloque con sangría



## Single Program, Multiple Data SPMD

1.   Elemento de lista
2.   Elemento de lista



This code forms the basis of all of the other examples that follow. It is the fundamental way we structure parallel programs today.


In [ ]:
%%writefile 00spmd.py
from mpi4py import MPI

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    print("Saludos desde el proceso {} de {} en {}"\
    .format(id, numProcesses, myHostName))

########## Run the main function
main()


Writing 00spmd.py


In [ ]:
! mpirun --allow-run-as-root -np 4 python 00spmd.py

Saludos desde el proceso 0 de 4 en 4afe557c9fa0
Saludos desde el proceso 3 de 4 en 4afe557c9fa0
Saludos desde el proceso 2 de 4 en 4afe557c9fa0
Saludos desde el proceso 1 de 4 en 4afe557c9fa0


The fundamental idea of message passing programs can be illustrated like this:

![picture](https://drive.google.com/uc?id=1wpQaFiaubIcQBV9Lw_jwOU0y2-K-EChW)

Cada proceso se configura dentro de una red de comunicación para poder comunicarse con todos los demás procesos a través de enlaces de comunicación. Cada proceso está configurado para tener su propio número, o id, que comienza en 0.

Nota: Cada proceso tiene sus propias copias de las 4 variables de datos anteriores. Entonces, aunque hay un solo programa, se ejecuta varias veces en procesos separados, cada uno con sus propios valores de datos. Esta es la razón del nombre del patrón que representa este código: programa único, datos múltiples. La línea de impresión al final de main () representa las múltiples salidas de datos diferentes producidas por cada proceso.


## Master-Worker
Este también es un patrón muy común que se usa en programación paralela y distribuida. Aquí está el pequeño código ilustrativo de muestra. Revísalo y responde esto: ¿Qué es diferente entre este ejemplo y el anterior?

In [ ]:
%%writefile 01masterWorker.py
from mpi4py import MPI

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    if id == 0:
        print("Saludos del maestro, {} de {} en {}"\
        .format(id, numProcesses, myHostName))
    else:
        print("Saludos del trabajador, {} de {} en {}"\
        .format(id, numProcesses, myHostName))

########## Run the main function
main()


Writing 01masterWorker.py


La respuesta a la pregunta anterior ilustra lo que podemos hacer con este patrón: basándonos en la identificación del proceso, podemos hacer que un proceso lleve a cabo algo diferente a los demás. Este concepto se utiliza mucho como un medio para coordinar actividades, donde un proceso, a menudo llamado maestro, tiene la responsabilidad de entregar el trabajo y realizar un seguimiento de los resultados. Veremos esto en ejemplos posteriores.

**Note:**
Por convención, el proceso de coordinación maestro suele ser el proceso número 0.

In [ ]:
! mpirun --allow-run-as-root -np 6 python 01masterWorker.py

Saludos del trabajador, 1 de 6 en 4ea9d432a738
Saludos del trabajador, 4 de 6 en 4ea9d432a738
Saludos del maestro, 0 de 6 en 4ea9d432a738
Saludos del trabajador, 2 de 6 en 4ea9d432a738
Saludos del trabajador, 3 de 6 en 4ea9d432a738
Saludos del trabajador, 5 de 6 en 4ea9d432a738


### Exercises:

- Vuelva a ejecutar, utilizando un número variable de procesos del 1 al 8 (es decir, varíe el argumento después de -np).

- Explique qué permanece igual y qué cambia a medida que cambia el número de procesos.

# Descomposición usando paralelismo para patrones de bucles

La forma más común de completar una tarea repetida en cualquier lenguaje de programa es un bucle. Usamos bucles porque queremos hacer un cierto número de tareas, muy a menudo porque queremos trabajar en un conjunto de elementos de datos que se encuentran en una lista o una matriz, o alguna otra estructura de datos. Si el trabajo a realizar en cada ciclo es independiente de las iteraciones anteriores, podemos usar procesos separados para hacer partes del ciclo de forma independiente. Este patrón de estructura de programa se denomina patrón de bucle para paralelizar, que es una estrategia de implementación para la descomposición del trabajo a realizar en partes más pequeñas.

## Parallel Loop Split into Equal Sized Chunks -> G1

In the code below, notice the use of the variable called `REPS`. This is designed to be the total amount or work, or repetitions, that the for loop is accomplishing. This particular code is designed so that if those repetitions do not divide equally by the number of processes, then the program will stop with a warning message printed by the master process.

Remember that because this is still also a SPMD program, all processes execute the code in the part of the if statement that evaluates to True. Each process has its own id, and we can determine how many processes there are, so we can choose where in the overall number of REPs of the loop each process will execute.

En el código siguiente, observe el uso de la variable llamada "REPS". Esto está diseñado para ser la cantidad total de trabajo, o repeticiones, que el ciclo for está logrando. Este código en particular está diseñado para que si esas repeticiones no se dividen equitativamente por el número de procesos, el programa se detendrá con un mensaje de advertencia impreso por el proceso maestro.

Recuerde que debido a que también es un programa SPMD, todos los procesos ejecutan el código en la parte de la instrucción if que se evalúa como Verdadero. Cada proceso tiene su propia identificación, y podemos determinar cuántos procesos hay, por lo que podemos elegir en qué parte del número total de REP del ciclo se ejecutará cada proceso.

In [ ]:
%%writefile 02parallelLoopEqualChunks.py
from mpi4py import MPI

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    REPS = 8

    if ((REPS % numProcesses) == 0 and numProcesses <= REPS):
        # How much of the loop should a process work on?
        chunkSize = int(REPS / numProcesses) #5
        start = id * chunkSize # 0 * 5 = 0 | 1 * 5 = 5
        stop = start + chunkSize # 0 + 5 = 5 | 5 + 5 = 10

        # do the work within the range set aside for this process
        for i in range(start, stop): # 0 a 4
            print("On {}: Process {} is performing iteration {}"\
            .format(myHostName, id, i)) # dsa3123, 0, 0 | dsa3123, 0, 1 | dsa3123, 0, 2 | dsa3123, 0, 3 |dsa3123, 0, 5

    else:
        # cannot break into equal chunks; one process reports the error
        if id == 0 :
            print("Please run with number of processes divisible by \
and less than or equal to {}.".format(REPS))

########## Run the main function
main()


Writing 02parallelLoopEqualChunks.py


In [ ]:
! mpirun --allow-run-as-root -np 4 python 02parallelLoopEqualChunks.py

On 4a2861f4dd33: Process 0 is performing iteration 0
On 4a2861f4dd33: Process 0 is performing iteration 1
On 4a2861f4dd33: Process 2 is performing iteration 4
On 4a2861f4dd33: Process 2 is performing iteration 5
On 4a2861f4dd33: Process 3 is performing iteration 6
On 4a2861f4dd33: Process 3 is performing iteration 7
On 4a2861f4dd33: Process 1 is performing iteration 2
On 4a2861f4dd33: Process 1 is performing iteration 3


### Ejercicios

- Run, using these numbers of processes, N: 1, 2, 4, and 8 (i.e., vary the  argument to -np).
- Change REPS to 16 in the code and rerun it. Then rerun with mpirun, varying N again.
- Explain how this pattern divides the iterations of the loop among the processes.

Which of the following is the correct assignment of loop iterations to processes for this code, when REPS is 8 and numProcesses is 4?

Ejecute, utilizando estos números de procesos, N: 1, 2, 4 y 8 (es decir, varíe el argumento a -np).
- Cambie REPS a 16 en el código y vuelva a ejecutarlo. Luego, vuelva a ejecutar con mpirun, variando N nuevamente.
- Explica cómo este patrón divide las iteraciones del bucle entre los procesos.

¿Cuál de las siguientes es la asignación correcta de iteraciones de bucle a procesos para este código, cuando REPS es 8 y numProcesses es 4?

![picture](https://drive.google.com/uc?id=1eUsjxYdWXWqThO_rdLO91HaLqBLAAh_S)

## Parallel for Loop Program Structure: chunks of 1

In the code below, we again use the variable called `REPS` for the total amount or work, or repetitions, that the for loop is accomplishing. This particular code is designed so that the number of repetitions should be more than or equal to the number of processes requested.
.. note:: Typically in real problems, the number of repetitions is much higher than the number of processes. We keep it small here to illustrate what is happening.

Like the last example all processes execute the code in the part of the if statement that evaluates to True. Note that in the for loop in this case we simply have process whose id is 0 start at iteration 0, then skip to 0 + numProcesses for its next iteration, and so on. Similarly, process 1 starts at iteration 1, skipping next to 1+ numProcesses, and continuing until REPs is reached. Each process performs similar single 'slices' or 'chunks of size 1' of the whole loop.

En el siguiente código, nuevamente usamos la variable llamada REPS para la cantidad total de trabajo, o repeticiones, que el ciclo for está logrando. Este código en particular está diseñado para que el número de repeticiones sea mayor o igual al número de procesos solicitados. .. nota :: Normalmente, en problemas reales, el número de repeticiones es mucho mayor que el número de procesos. Lo mantenemos pequeño aquí para ilustrar lo que está sucediendo.

Como en el último ejemplo, todos los procesos ejecutan el código en la parte de la instrucción if que se evalúa como Verdadero. Tenga en cuenta que en el bucle for en este caso simplemente tenemos un proceso cuyo id es 0, comienza en la iteración 0, luego salta a 0 + numProcesses para su próxima iteración, y así sucesivamente. De manera similar, el proceso 1 comienza en la iteración 1, saltando junto a 1+ numProcesses y continuando hasta que se alcanzan los REP. Cada proceso realiza 'rebanadas' individuales similares o 'trozos de tamaño 1' de todo el ciclo


In [ ]:
%%writefile 03parallelLoopChunksOf1.py
from mpi4py import MPI

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    REPS = 8

    if (numProcesses <= REPS): #4<8

        for i in range(id, REPS, numProcesses): # i (0,8,4) | (1,8,4)
            print("On {}: Process {} is performing iteration {}"\
            .format(myHostName, id, i)) #nrsdy656, 0, 0 | nrsdy656, 0, 4 |ssfs, 1, 1 | ssfs, 1, 5|

    else:
        # can't have more processes than work; one process reports the error
        if id == 0 :
            print("Please run with number of processes less than \
or equal to {}.".format(REPS))

########## Run the main function
main()


Overwriting 03parallelLoopChunksOf1.py


In [ ]:
! mpirun --allow-run-as-root -np 4 python 03parallelLoopChunksOf1.py

On 4a2861f4dd33: Process 1 is performing iteration 1
On 4a2861f4dd33: Process 2 is performing iteration 2
On 4a2861f4dd33: Process 2 is performing iteration 6
On 4a2861f4dd33: Process 0 is performing iteration 0
On 4a2861f4dd33: Process 1 is performing iteration 5
On 4a2861f4dd33: Process 3 is performing iteration 3
On 4a2861f4dd33: Process 3 is performing iteration 7
On 4a2861f4dd33: Process 0 is performing iteration 4


Para determinar el trabajo que va a realizar cada proceso, se tiene:

1.  NroIteracionesPrimerosProcesos = REPS DIV NumProcesses + 1 | Llamamos PrimerosProcesos a los que van de 0 a (REPS - (REPS DIV NumProcesses) * NumProcesses - 1).
2.  NroIteracionesUltimosProcesos = REPS DIV NumProcesses | Llamamos UltimosProcesos a los que van de (REPS - (REPS DIV NumProcesses) * NumProcesses) a (NumProcesses - 1).

Por ejemplo: REPS = 8 & NumProcesses = 3

Entonces: 

1.   NroIteracionesPrimerosProcesos = 8 DIV 3 + 1 = 3 || PrimerosProcesos van de 0 a (8 - (8 DIV 3) * 3 - 1 = 1).

    Es decir, los primeros procesos van de 0 a 1 y cada uno hará 3 iteraciones

     
2.   NroIteracionesUltimosProcesos = 8 DIV 3 = 2 || UltimosProcesos van de (8 - (8 DIV 3) * 3 = 2) a (3 - 1 = 2).

    Es decir, los últimos procesos van de 2 a 2 (solo el 2)y cada uno hará 2 iteraciones

By: Jeremy Axl Lazo Mendoza





### Exercises
- Run, using these numbers of processes, N: 1, 2, 4, and 8
- Compare source code to output.
- Change REPS to 16, save, rerun, varying N again.
- Explain how this pattern divides the iterations of the loop among the processes.

Which of the following is the correct assignment of loop iterations to processes for this code, when REPS is 8 and numProcesses is 4?


![picture](https://drive.google.com/uc?id=1eUsjxYdWXWqThO_rdLO91HaLqBLAAh_S)

# Point to point communication: the message passing pattern

The fundamental basis of coordination between independent processes is point-to-point communication between processes through the communication links in the MPI.COMM_WORLD. The form of communication is called message passing, where one process **sends** data to another one, who in turn must **receive** it from the sender. This is illustrated as follows:

![picture](https://drive.google.com/uc?id=1WJcOXq6Dn5TKF9Lng8r18_y2b23tHFe8)

## Message Passing Pattern: Key Problem

The following code represents a common error that many programmers have inadvertently placed in their code. The concept behind this program is that we wish to use communication between pairs of processes, like this:

![picture](https://drive.google.com/uc?id=1UJ2acj6XzphD2W6gnutNF2YGp6wU529Z)

For message passing to work between a pair of processes, one must send and the other must receive. If we wish to **exchange** data, then each process will need to perform both a send and a receive.
The idea is that process 0 will send data to process 1, who will receive it from process 0. Process 1 will also send some data to process 0, who will receive it from process 1. Similarly, processes 2 and 3 will exchange messages: process 2 will send data to process 3, who will receive it from process 2. Process 3 will also send some data to process 2, who will receive it from process 3.

If we have more processes, we still want to pair up processes together to exchange messages. The mechanism for doing this is to know your process id. If your id is odd (1, 3 in the above diagram), you will send and receive from your neighbor whose id is id - 1. If your id is even (0, 2), you will send and receive from your neighbor whose id is id + 1. This should work even if we add more than 4 processes, as long as the number of processes is divisible by 2.

![warning sign](https://drive.google.com/uc?id=1SEqDBTBSKwNVXzn-zueWa7fBCm5b1_MB)
**Warning** There is a problem with the following code called *deadlock*. This happens when every process is waiting on an action from another process. The program cannot complete. **To stop the program, choose the small square that appears after you choose to run the mpirun cell.**


In [13]:
%%writefile 04messagePssingDeadlock.py
from mpi4py import MPI

# function to return whether a number of a process is odd or even
def odd(number):
    if (number % 2) == 0:
        return False
    else :
        return True

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    if numProcesses > 1 and not odd(numProcesses):
        sendValue = id
        if odd(id):
            #odd processes receive from their paired 'neighbor', then send
            comm.send(sendValue, dest=id-1)
            receivedValue = comm.recv(source=id-1)
            
        else :
            #even processes receive from their paired 'neighbor', then send
            comm.send(sendValue, dest=id+1)
            receivedValue = comm.recv(source=id+1)

        print("Process {} of {} on {} computed {} and received {}"\
        .format(id, numProcesses, myHostName, sendValue, receivedValue))

    else :
        if id == 0:
            print("Please run this program with the number of processes \
positive and even")

########## Run the main function
main()


Overwriting 04messagePssingDeadlock.py


In [16]:
! mpirun --allow-run-as-root -np 6 python 04messagePssingDeadlock.py

Process 4 of 6 on 59784989e831 computed 4 and received 5
Process 5 of 6 on 59784989e831 computed 5 and received 4
Process 3 of 6 on 59784989e831 computed 3 and received 2
Process 2 of 6 on 59784989e831 computed 2 and received 3
Process 0 of 6 on 59784989e831 computed 0 and received 1
Process 1 of 6 on 59784989e831 computed 1 and received 0


![warning sign](https://drive.google.com/uc?id=1SEqDBTBSKwNVXzn-zueWa7fBCm5b1_MB)Remember,**To stop the program, choose the small square that appears after you choose to run the mpirun cell.**

#### What causes the deadlock?

Each process, regardless of its id, will execute a receive request first. In this model, recv is a **blocking** function- it will not continue until it gets data from a send. So every process is blocked waiting to receive a message.

#### Can you think of how to fix this problem?

Since recv is a **blocking** function, we need to have some processes send first, while others correspondingly recv first from those who send first. This provides coordinated exchanges.

Go to the next example to see the solution.


## Message Passing Patterns: avoiding deadlock

Let's look at a few more correct message passing examples.

### Fix the Deadlock

To fix deadlock of the previous example, we coordinate the communication between pairs of processes so that there is an ordering of sends and receives between them.

![Important symbol](https://drive.google.com/uc?id=1AWRLAqeaqi7SG7PHyOVywZRuMDK9Z2_s)**Important:** The new code corrects deadlock with a simple change: odd process sends first, even process receives first. *This is the proper pattern for exchanging data between pairs of processes.*

In [ ]:
%%writefile 05messagePassing.py
from mpi4py import MPI

# function to return whether a number of a process is odd or even
def odd(number):
    if (number % 2) == 0:
        return False
    else :
        return True

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    if numProcesses > 1 and not odd(numProcesses):
        sendValue = id
        if odd(id):
            #odd processes send to their paired 'neighbor', then receive from
            comm.send(sendValue, dest=id-1)
            receivedValue = comm.recv(source=id-1)
        else :
            #even processes receive from their paired 'neighbor', then send
            receivedValue = comm.recv(source=id+1)
            comm.send(sendValue, dest=id+1)

        print("Process {} of {} on {} computed {} and received {}"\
        .format(id, numProcesses, myHostName, sendValue, receivedValue))

    else :
        if id == 0:
            print("Please run this program with the number of processes \
positive and even")

########## Run the main function
main()


Overwriting 05messagePassing.py


In [ ]:
! mpirun --allow-run-as-root -np 6 python 05messagePassing.py

Process 2 of 6 on 4afe557c9fa0 computed 2 and received 3
Process 3 of 6 on 4afe557c9fa0 computed 3 and received 2
Process 0 of 6 on 4afe557c9fa0 computed 0 and received 1
Process 4 of 6 on 4afe557c9fa0 computed 4 and received 5
Process 1 of 6 on 4afe557c9fa0 computed 1 and received 0
Process 5 of 6 on 4afe557c9fa0 computed 5 and received 4


### Exercise

- Run, using N = 4, 6, 8, and 10 processes. (Note what happens if you use an odd number instead.)


## Sending data structures
This next example illustrates that we can exchange different lists of data between processes.


In [19]:
%%writefile 06messagePassing2.py
from mpi4py import MPI

# function to return whether a number of a process is odd or even
def odd(number):
    if (number % 2) == 0:
        return False
    else :
        return True

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    if numProcesses > 1 and not odd(numProcesses):
        #generate a list of 8 numbers, beginning with my id
        sendList = list(range(id, id+8))
        if odd(id):
            #odd processes send to their 'left neighbor', then receive from
            comm.send(sendList, dest=id-1)
            receivedList = comm.recv(source=id-1)
        else :
            #even processes receive from their 'right neighbor', then send
            receivedList = comm.recv(source=id+1)
            comm.send(sendList, dest=id+1)

        print("Process {} of {} on {} computed {} and received {}"\
        .format(id, numProcesses, myHostName, sendList, receivedList))

    else :
        if id == 0:
            print("Please run this program with the number of processes \
positive and even")

########## Run the main function
main()

Overwriting 06messagePassing2.py


In [18]:
! mpirun --allow-run-as-root -np 4 python 06messagePassing2.py

Process 2 of 4 on 59784989e831 computed [2, 3, 4, 5, 6, 7, 8, 9] and received [3, 4, 5, 6, 7, 8, 9, 10]
Process 3 of 4 on 59784989e831 computed [3, 4, 5, 6, 7, 8, 9, 10] and received [2, 3, 4, 5, 6, 7, 8, 9]
Process 0 of 4 on 59784989e831 computed [0, 1, 2, 3, 4, 5, 6, 7] and received [1, 2, 3, 4, 5, 6, 7, 8]
Process 1 of 4 on 59784989e831 computed [1, 2, 3, 4, 5, 6, 7, 8] and received [0, 1, 2, 3, 4, 5, 6, 7]


### Exercise

- Run, using N = 4, 6, 8, and 10 processes. 
- In the above code, locate where the list of elements to be sent is being made by each process. What is different about each list per process?


## Ring of passed messages
Another pattern that appears in message passing programs is to use a ring of processes, where messages get sent in this fashion:

![picture of ring of message passing](https://drive.google.com/uc?id=16VMF9t8nD3JcVehFvs4dbzIiU5eDuZbG)

When we have 4 processes, the idea is that process 0 will send data to process 1, who will receive it from process 0 and then send it to process 2, who will receive it from process 1 and then send it to process 3, who will receive it from process 2 and then send it back around to process 0.

In [20]:
%%writefile 07messagePassing3.py
from mpi4py import MPI

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    if numProcesses > 1 :

        if id == 0:        # master
            #generate a list with master id in it
            sendList = [id]
            # send to the first worker
            comm.send(sendList, dest=id+1)
            print("Master Process {} of {} on {} sent {}"\
            .format(id, numProcesses, myHostName, sendList))
            # receive from the last worker
            receivedList = comm.recv(source=numProcesses-1)
            print("Master Process {} of {} on {} received {}"\
            .format(id, numProcesses, myHostName, receivedList))
        else :
            # worker: receive from any source
            receivedList = comm.recv(source=id-1)
            # add this worker's id to the list and send along to next worker,
            # or send to the master if the last worker
            sendList = receivedList + [id]
            comm.send(sendList, dest=(id+1) % numProcesses)

            print("Worker Process {} of {} on {} received {} and sent {}"\
            .format(id, numProcesses, myHostName, receivedList, sendList))

    else :
        print("Please run this program with the number of processes \
greater than 1")

########## Run the main function
main()


Writing 07messagePassing3.py


In [22]:
! mpirun --allow-run-as-root -np 3 python 07messagePassing3.py

Master Process 0 of 3 on 59784989e831 sent [0]
Worker Process 2 of 3 on 59784989e831 received [0, 1] and sent [0, 1, 2]
Worker Process 1 of 3 on 59784989e831 received [0] and sent [0, 1]
Master Process 0 of 3 on 59784989e831 received [0, 1, 2]


### Exercises
- Run, using N = from 1 through 8 processes.
- Make sure that you can trace how the code generates the output that you see.
- How is the finishing of the 'ring' completed, where the last process determines that it should send back to process 0?

# Collective Communication: Broadcast pattern
There are many cases when a master process obtains or creates data that needs to be sent to all of the other processes. There is a special pattern for this called **broadcast**. You will see examples of the master sending different types of data to each of the other processes.

## Broadcast from master to workers

We will look at three types of data that can be created in the master and sent to the workers. Rather than use send and receive, we will use a special new function called bcast.

![Important symbol](https://drive.google.com/uc?id=1AWRLAqeaqi7SG7PHyOVywZRuMDK9Z2_s) **Note:** In each code example, note how the master does one thing, and the workers do another, but **all of the processes execute the bcast function.**


### Broadcast a dictionary

Find the place in this code where the data is being broadcast to all of the processes. Match the prints to the output you observe when you run it.

In [ ]:
%%writefile 08broadcast.py
from mpi4py import MPI

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    if numProcesses > 1 :

        if id == 0:        # master
            #master: generate a dictionary with arbitrary data in it
            data = {'one': 1, 'two': 2, 'three': 3}
            print("Master Process {} of {} on {} broadcasts {}"\
            .format(id, numProcesses, myHostName, data))

        else :
            # worker: start with empty data
            data = {}
            print("Worker Process {} of {} on {} starts with {}"\
            .format(id, numProcesses, myHostName, data))

        #initiate and complete the broadcast
        data = comm.bcast(data, root=0)
        #check the result
        print("Process {} of {} on {} has {} after the broadcast"\
        .format(id, numProcesses, myHostName, data))

    else :
        print("Please run this program with the number of processes \
greater than 1")

########## Run the main function
main()


Writing 08broadcast.py


In [ ]:
! mpirun --allow-run-as-root -np 4 python 08broadcast.py

Worker Process 2 of 4 on 4afe557c9fa0 starts with {}
Worker Process 3 of 4 on 4afe557c9fa0 starts with {}
Master Process 0 of 4 on 4afe557c9fa0 broadcasts {'one': 1, 'two': 2, 'three': 3}
Worker Process 1 of 4 on 4afe557c9fa0 starts with {}
Process 0 of 4 on 4afe557c9fa0 has {'one': 1, 'two': 2, 'three': 3} after the broadcast
Process 1 of 4 on 4afe557c9fa0 has {'one': 1, 'two': 2, 'three': 3} after the broadcast
Process 3 of 4 on 4afe557c9fa0 has {'one': 1, 'two': 2, 'three': 3} after the broadcast
Process 2 of 4 on 4afe557c9fa0 has {'one': 1, 'two': 2, 'three': 3} after the broadcast


#### Exercise
- Run, using N = from 1 through 8 processes.

### Broadcast user input

The following program will take extra input that will get broadcast to all processes.

In [ ]:
%%writefile 09broadcastUserInput.py
from mpi4py import MPI
import sys

# Determine if user provided a string to be broadcast.
# If not, quit with a warning.
def checkInput(id):
    numArguments = len(sys.argv)
    if numArguments == 1:
        #no extra argument was given- master warns and all exit
        if id == 0:
            print("Please add a string to be broadcast from master to workers")
        sys.exit()

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    if numProcesses > 1 :
        checkInput(id)

        if id == 0:        # master
            #master: get the command line argument
            data = sys.argv[1]
            print("Master Process {} of {} on {} broadcasts \"{}\""\
            .format(id, numProcesses, myHostName, data))

        else :
            # worker: start with empty data
            data = 'No data'
            print("Worker Process {} of {} on {} starts with \"{}\""\
            .format(id, numProcesses, myHostName, data))

        #initiate and complete the broadcast
        data = comm.bcast(data, root=0)
        #check the result
        print("Process {} of {} on {} has \"{}\" after the broadcast"\
        .format(id, numProcesses, myHostName, data))

    else :
        print("Please run this program with the number of processes \
greater than 1")

########## Run the main function
main()

Writing 09broadcastUserInput.py


![warning sign](https://drive.google.com/uc?id=1SEqDBTBSKwNVXzn-zueWa7fBCm5b1_MB)
**Warning** This program is unlike any of the others and takes in a second argument, as shown below. 

In [ ]:
! mpirun --allow-run-as-root -np 4 python 09broadcastUserInput.py "hello world!"

Worker Process 1 of 4 on 4afe557c9fa0 starts with "No data"
Worker Process 2 of 4 on 4afe557c9fa0 starts with "No data"
Worker Process 3 of 4 on 4afe557c9fa0 starts with "No data"
Master Process 0 of 4 on 4afe557c9fa0 broadcasts "hello world!"
Process 0 of 4 on 4afe557c9fa0 has "hello world!" after the broadcast
Process 1 of 4 on 4afe557c9fa0 has "hello world!" after the broadcast
Process 2 of 4 on 4afe557c9fa0 has "hello world!" after the broadcast
Process 3 of 4 on 4afe557c9fa0 has "hello world!" after the broadcast


#### Exercise
- Run, using N = from 1 through 8 processes, with a string of your choosing.

### Broadcast a list

This is just one more example to show that other data structures can also be broadcast from the master to all worker processes.

In [ ]:
%%writefile 11broadcastList.py
from mpi4py import MPI

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    if numProcesses > 1 :

        if id == 0:        # master
            #master: generate a dictionary with arbitrary data in it
            data = list(range(numProcesses))
            print("Master Process {} of {} on {} broadcasts {}"\
            .format(id, numProcesses, myHostName, data))

        else :
            # worker: start with empty data
            data = []
            print("Worker Process {} of {} on {} starts with {}"\
            .format(id, numProcesses, myHostName, data))

        #initiate and complete the broadcast
        data = comm.bcast(data, root=0)

        #check the result
        print("Process {} of {} on {} has {} after the broadcast"\
        .format(id, numProcesses, myHostName, data))

    else :
        print("Please run this program with the number of processes greater than 1")

########## Run the main function
main()



Writing 11broadcastList.py


In [ ]:
! mpirun --allow-run-as-root -np 4 python 11broadcastList.py

Worker Process 2 of 4 on 4afe557c9fa0 starts with []
Worker Process 3 of 4 on 4afe557c9fa0 starts with []
Master Process 0 of 4 on 4afe557c9fa0 broadcasts [0, 1, 2, 3]
Process 0 of 4 on 4afe557c9fa0 has [0, 1, 2, 3] after the broadcast
Worker Process 1 of 4 on 4afe557c9fa0 starts with []
Process 1 of 4 on 4afe557c9fa0 has [0, 1, 2, 3] after the broadcast
Process 2 of 4 on 4afe557c9fa0 has [0, 1, 2, 3] after the broadcast
Process 3 of 4 on 4afe557c9fa0 has [0, 1, 2, 3] after the broadcast


#### Exercise
- Run, using N = from 1 through 8 processes.


# Collective Communication: reduction pattern

There are often cases when every process needs to complete a partial result of an overall computation. For example if you want to process a large set of numbers by summing them together into one value (i.e. *reduce* a set of numbers into one value, its sum), you could do this faster by having each process compute a partial sum, then have all the processes communicate to add each of their partial sums together.

This is so common in parallel processing that there is a special collective communication function called **reduce** that does just this.

## Collective Communication: reduce function

The type of reduction of many values down to one can be done with different types of operators on the set of values computed by each process.


### Reduce all values using sum and max
In this example, every process computes the square of (id+1). Then all those values are summed together and also the maximum function is applied.

In [ ]:
%%writefile 12reduction.py
from mpi4py import MPI

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    square = (id+1) * (id+1)

    if numProcesses > 1 :
        #initiate and complete the reductions
        sum = comm.reduce(square, op=MPI.SUM)
        max = comm.reduce(square, op=MPI.MAX)
    else :
        sum = square
        max = square

    if id == 0:        # master/root process will print result
        print("The sum of the squares is  {}".format(sum))
        print("The max of the squares is  {}".format(max))

########## Run the main function
main()

In [ ]:
! mpirun --allow-run-as-root -np 4 python 12reduction.py

#### Exercises
- Run, using N = from 1 through 8 processes.
- Try replacing MPI.MAX with MPI.MIN(minimum) and/or replacing MPI.SUM with MPI.PROD (product). Then save and run the code again.


### Reduction on a list of values

We can try reduction with lists of values, but the behavior matches Python semantics regarding lists. Note this in the following example. Then note how you can change the semantics in the exercises.


In [ ]:
%%writefile 13reductionList.py
from mpi4py import MPI

# Exercise: Can you explain what this function returns,
#           given two lists as input?
def sumListByElements(x,y):
    return [a+b for a, b in zip(x, y)]

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    srcList = [1*id, 2*id, 3*id, 4*id, 5*id]

    destListMax = comm.reduce(srcList, op=MPI.MAX)
    destListSum = comm.reduce(srcList, op=MPI.SUM)
    #destListSumByElement = comm.reduce(srcList, op=sumListByElements)

    if id == 0:        # master/root process will print result
        print("The resulting reduce max list is  {}".format(destListMax))
        print("The resulting reduce sum list is  {}".format(destListSum))
        #print("The resulting reduce sum list is  {}".format(destListSumByElement))

########## Run the main function
main()


In [ ]:
! mpirun --allow-run-as-root -np 4 python 13reductionList.py

#### Exercises
- Run, using N = from 1 through 4 processes.
- Uncomment the two lines of runnable code that are commented in the main() function. Observe the new results and explain why the MPI.SUM (using the + operator underneath) behaves the way it does on lists, and what the new function called sumListByElements is doing instead.

![Important symbol](https://drive.google.com/uc?id=1AWRLAqeaqi7SG7PHyOVywZRuMDK9Z2_s) **Note:** There are two ways in Python that you might want to sum a set of lists from each process: 1) concatenating the elements together, or 2) summing the element at each location from each process and placing the sum in that location in a new list. In the latter case, the new list is the same length as the original lists on each process.


# Collective Communication: scatter and gather pattern

There are often cases when each process can work on some portion of a larger data structure. This can be carried out by having the master process maintain the larger structure and send parts to each of the worker processes, keeping part of the structure on the master. Each process then works on their portion of the data, and then the master can get the completed portions back.

This is so common in message passing parallel processing that there are two special collective communication functions called **scatter** and **gather** that handle this.


## Collective Communication: scatter and gather lists

When several processes need to work on portions of a data structure, such as a list of lists or a 1-d or 2-d array, at various points in a program, a way to do this is to have one node, usually the master, divide the data structure and send portions to each of the other processes, often keeping one portion for itself. Each process then works on that portion of the data, and then the master can get the completed portions back. This type of coordination is so common that MPI has special patterns for it called **scatter** and **gather**.


### Scatter Lists
The following diagrams illustrate how scatter using python list structures works. The master contains a list of lists and all processes participate in the scatter:

![scatter lists diagram](https://drive.google.com/uc?id=1QDRW2JeAa_TelKxZTphCPF393Bxn_BbL)

After the scatter is completed, each process has one of the smaller lists to work on, like this:

![after scatter lists diagram](https://drive.google.com/uc?id=1xA2NRtm1k4_g16tJTWBFCVArKLfIEurc)

In this next code example, some small lists are created in a list whose length is as long as the number of processes.

![Important symbol](https://drive.google.com/uc?id=1AWRLAqeaqi7SG7PHyOVywZRuMDK9Z2_s) **Note:** In the code below, note how all processes must call the scatter function.

In [ ]:
%%writefile 14scatter.py
from mpi4py import MPI

# Create a list of lists to be scattered.
def genListOfLists(numElements):
    data = [[0]*3 for i in range(numElements)]
    for i in range(numElements):
        #make small lists of 3 distinct elements
        smallerList = []
        for j in range(1,4):
            smallerList = smallerList + [(i+1)*j]
        # place the small list in the larger list
        data[i] = smallerList
    return data

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    # in mpi4py, the lowercase scatter method only works on lists whose size
    # is the total number of processes.
    numElements = numProcesses      #total elements in list created by master process

    # however, the list can contain lists, like this list of 3-element lists,
    # for example this list of four 3-element lists:
    #     [[1, 2, 3], [2, 4, 6], [3, 6, 9], [4, 8, 12]]

    if id == 0:
        data = genListOfLists(numElements)
        print("Master {} of {} on {} has created list: {}"\
        .format(id, numProcesses, myHostName, data))
    else:
        data = None
        print("Worker Process {} of {} on {} starts with {}"\
        .format(id, numProcesses, myHostName, data))

    #scatter one small list in the large list on node 0 to each of the processes
    result = comm.scatter(data, root=0)

    print("Process {} of {} on {} has result after scatter {}"\
    .format(id, numProcesses, myHostName, result))

    if id == 0:
        print("Master {} of {} on {} has original list after scatter: {}"\
        .format(id, numProcesses, myHostName, data))

########## Run the main function
main()


In [ ]:
! mpirun --allow-run-as-root -np 4 python 14scatter.py

#### Exercises
- Run, using N = from 2 through 8 processes.
- If you want to study the code, explain to yourself what genListofLists does in the code below.


### Gather Lists
Once several processes have their own lists of data, those lists can also be gathered back together into a list of lists, usually in the master process. All processes participate in a gather, like this:

![before gather diagram](https://drive.google.com/uc?id=1OWHNMKCEKsGpExJCO6l5czW9QFyMZiT6)

The gather creates a list of lists in the master, like this:

![after gather diagram](https://drive.google.com/uc?id=1W9lky1LY0L0K6iyA00jsNV4hAnmmbvP2)

In this example, each process creates some very small lists. Then a gather is used to create a list of lists on the master process.

![Important symbol](https://drive.google.com/uc?id=1AWRLAqeaqi7SG7PHyOVywZRuMDK9Z2_s) **Note:** In the code below, note how all processes must call the gather function.


In [ ]:
%%writefile 15gather.py
from mpi4py import MPI

SMALL_LIST_SIZE = 3

# create a small list whose values contain id times multiples of 10
def genSmallList(id):
    smallerList = []
    for j in range(1, SMALL_LIST_SIZE+1):
        smallerList = smallerList + [(id * 10)*j]
    return smallerList

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    #all processes create small lists
    sendData = genSmallList(id)
    print("Process {} of {} on {} starts with {}"\
    .format(id, numProcesses, myHostName, sendData))

    # gather the small lists at the master node:
    # final result is a list whose length == the number of processes
    result = comm.gather(sendData, root=0)

    # only the master node has all of the small lists
    if id == 0:
        print("Process {} of {} on {} has result after gather {}"\
        .format(id, numProcesses, myHostName, result))

########## Run the main function
main()

In [ ]:
! mpirun --allow-run-as-root -np 4 python 15gather.py

#### Exercises
- Run, using N = from 2 through 8 processes.
- Try with different values of SMALL_LIST_SIZE, perhaps changing printing of result for readability


## Collective Communication:  scatter and gather arrays

The mpi4py library of functions has several collective communication functions that are designed to work with arrays created using the python library for numerical analysis computations called *numpy*.

If you are unfamiliar with using numpy, and want to know more about its features and available methods, you will need to consult another tutorial for that. It should be possible to understand the following scatter, then gather example by observing the results that get printed, even if you are unfamiliar with the functions from numpy that are used to create the 1-D array.

The numpy library has special data structures called arrays, that are common in other programming languages. A 1-dimensional array of integers can be envisioned very much like a list of integers, where each value in the array is at a particular index. The mpi4py Scatter function, with a capital S, can be used to send portions of a larger array on the master to the workers, like this:

![alt text](https://drive.google.com/uc?id=1n2YmY12tBrTxtJK6MFpBX9nWmopQGT_s)

The result of doing this then looks like this, where each process has a portion of the original that they can then work on:

![alt text](https://drive.google.com/uc?id=19GNbTWWJEOU16wNjwzHon4jpC5fXKj_1)

The reverse of this process can be done using the Gather function.

In this example, a 1-D array is created by the master, then scattered, using Scatter (capital S). After each smaller array used by each process is changed, the Gather (capital G) function brings the full array with the changes back into the master.

![Important symbol](https://drive.google.com/uc?id=1AWRLAqeaqi7SG7PHyOVywZRuMDK9Z2_s) **Note:** In the code below, note how all processes must call the Scatter and Gather functions.

In [ ]:
%%writefile 16ScatterGather.py
from mpi4py import MPI
import numpy as np


# Create a 1D array to be scattered.
def genArray(numProcesses, numElementsPerProcess):

    data = np.linspace(1, #start
                numProcesses*numElementsPerProcess, #stop
                numElementsPerProcess*numProcesses, #total elements
                dtype='u4')  # 4-byte unsigned integer data type
    return data

def timesTen(a):
    return(a*10);

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    # in mpi4py, the uppercase Scatter method works on arrays generated by
    # numpy routines.
    #
    # Here we will create a single array designed to then scatter 3 elements
    # of it in a smaller array to each process.

    numDataPerProcess = 3

    if id == 0:
        data = genArray(numProcesses, numDataPerProcess)
        #genListOfLists(numElements)
        print("Master {} of {} on {} has created array: {}"\
        .format(id, numProcesses, myHostName, data))
    else:
        data = None
        print("Worker Process {} of {} on {} starts with {}"\
        .format(id, numProcesses, myHostName, data))

    #scatter one small array from a part of the large array
    # on node 0 to each of the processes
    smallerPart = np.empty(numDataPerProcess, dtype='u4') # allocate space for result on each process
    comm.Scatter(data, smallerPart, root=0)

    if id == 0:
        print("Master {} of {} on {} has original array after Scatter: {}"\
        .format(id, numProcesses, myHostName, data))

    print("Process {} of {} on {} has smaller part after Scatter {}"\
    .format(id, numProcesses, myHostName, smallerPart))

    # do some work on each element
    newValues = timesTen(smallerPart)

    print("Process {} of {} on {} has smaller part after work {}"\
    .format(id, numProcesses, myHostName, newValues))

    # All processes participate in gathering each of their parts back at
    # process 0, where the original data is now overwritten with new values
    # from eqch process.
    comm.Gather(newValues, data, root=0)

    if id == 0:
        print("Master {} of {} on {} has new data array after Gather:\n {}"\
        .format(id, numProcesses, myHostName, data))


########## Run the main function
main()


In [ ]:
! mpirun --allow-run-as-root -np 4 python 16ScatterGather.py

#### Exercises
- Run, using N = from 2 through 8 processes.
- If you want to study the numpy part of the code, look up the numpy function linspace used in genArray().


# When amount of work varies: balancing the load

There are algorithms where the master is used to assign tasks to workers by sending them data and receiving results back as each worker completes a task (or after the worker completes all of its tasks). In many of these cases, the computation time needed by each worker process for each of its tasks can vary somewhat dramatically. This situation is where **dynamic load balancing** can be helpful.

In this example we combine the master-worker pattern with message passing. The master has many tasks that need to be completed. The master starts by sending some data needed to complete a task to each worker process. Then the master loops and waits to hear back from each worker by receiving a message from any of them. When the master receives a message from a worker, it sends that worker more data for its next task, unless there are no more tasks to complete, in which case it sends a special message to the worker to stop running.

In this simple example, each worker is sent the number of seconds it should 'sleep', which can vary from 1 to 8. This illustrates varying sizes of workloads. Because of the code's simplicity, the number of tasks each worker does doesn't vary by much. In some real examples, the time for one task my be quite different than the time for another, which could have a different outcome, in which some workers were able to complete more tasks as others were doing long ones.

This approach can sometimes be an improvement on the assignment of an equal number of tasks to all processes.

Note in this case how the master, whose id is 0, handles the assignment of tasks, while the workers simply do what they are sent until they are told to stop.

In [ ]:
%%writefile 17dynamicLoadBalance.py
from mpi4py import MPI
import numpy as np
import time

def genTasks(numTasks):
    np.random.seed(1000)  # run the same set of timed tasks
    return np.random.randint(low=1, high=9, size=numTasks)

# tags that can be applied to messages
WORKTAG = 1
DIETAG = 2

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    if (id == 0) :
        # create an arbitrary array of numbers for how long each
        # worker task will 'work', by sleeping that amount of seconds
        numTasks = (numProcesses-1)*4 # avg 4 tasks per worker process
        workTimes = genTasks(numTasks)
        print("master created {} values for sleep times:".format(workTimes.size), flush=True)
        print(workTimes, flush=True)
        handOutWork(workTimes, comm, numProcesses)
    else:
        worker(comm)

def handOutWork(workTimes, comm, numProcesses):
    totalWork = workTimes.size
    workcount = 0
    recvcount = 0
    print("master sending first tasks", flush=True)
    # send out the first tasks to all workers
    for id in range(1, numProcesses):
        if workcount < totalWork:
            work=workTimes[workcount]
            comm.send(work, dest=id, tag=WORKTAG)
            workcount += 1
            print("master sent {} to {}".format(work, id), flush=True)

    # while there is still work,
    # receive result from a worker, which also
    # signals they would like some new work
    while (workcount < totalWork) :
        # receive next finished result
        stat = MPI.Status()
        workTime = comm.recv(source=MPI.ANY_SOURCE, status=stat)
        recvcount += 1
        workerId = stat.Get_source()
        print("master received {} from {}".format(workTime, workerId), flush=True)
        #send next work
        comm.send(workTimes[workcount], dest=workerId, tag=WORKTAG)
        workcount += 1
        print("master sent {} to {}".format(work, workerId), flush=True)

    # Receive results for outstanding work requests.
    while (recvcount < totalWork):
        stat = MPI.Status()
        workTime = comm.recv(source=MPI.ANY_SOURCE, status=stat)
        recvcount += 1
        workerId = stat.Get_source()
        print("end: master received {} from {}".format(workTime, workerId), flush=True)

    # Tell all workers to stop
    for id in range(1, numProcesses):
        comm.send(-1, dest=id, tag=DIETAG)


def worker(comm):
    # keep receiving messages and do work, unless tagged to 'die'
    while(True):
        stat = MPI.Status()
        waitTime = comm.recv(source=0, tag=MPI.ANY_TAG, status=stat)
        print("worker {} got {}".format(comm.Get_rank(), waitTime), flush=True)
        if (stat.Get_tag() == DIETAG):
            print("worker {} dying".format(comm.Get_rank()), flush=True)
            return
        # simulate work by sleeping
        time.sleep(waitTime)
        # indicate done with work by sending to Master
        comm.send(waitTime, dest=0)

########## Run the main function
main()


In [ ]:
! mpirun --allow-run-as-root -np 4 python 17dynamicLoadBalance.py

## Exercises
- Run, using N = 4 processes
- Study the execution carefully. Note that with 4 processes, 3 are workers. The total number of tasks is 3*4, or 12. Which process does the most work? You can count by looking for the lines that end with "... from X", where X is a worker process id.
- Try with N = 8 (7 workers).